First, we import the three modules necessary in this notebook.

In [3]:
import pymrio
import pandas as pd
import pickle

- pandas and numpy are libraries focused on data analysis
- pickle allows to save and export data produced from notebook to notebook
- pymrio is a library that transforms the .CSV files of EXIOBASE into pandas.dataframe, easier to manipulate

You'll need to download a version of EXIOBASE3 to use it. Fear not! It's free. You just need to register here: https://www.exiobase.eu/ and extract the base year you prefer (in this project we use 2011) and make sure to choose the pxp version (and not the ixi version).


Then enter the path to your **unzipped** EXIOBASE folder below:

In [4]:
#path_to_exiobase_unzipped_folder = 'my_own_path/IOT_2011_pxp/'
path_to_exiobase_unzipped_folder = 'C://Users/Maxime/Desktop/Thesis/Bdd/EXIOBASE/monetary/exiobase3.4/IOT_2011_pxp/'
io = pymrio.parse_exiobase3(path_to_exiobase_unzipped_folder)

We make a copy of the technology matrix of EXIOBASE.

In [53]:
extended_exio = io.A.copy()

We copy the rows and columns of Canada ('CA') and add them to the extended_exio dataframe

In [54]:
extended_exio = pd.concat([extended_exio, extended_exio.loc[:,[i for i in io.A.columns if i[0] == 'CA']]],axis=1)
extended_exio = pd.concat([extended_exio, extended_exio.loc[[i for i in io.A.columns if i[0] == 'CA'],:]],axis=0)

We rename the last set of products to 'QC' instead of having two 'CA'.

In [55]:
countries = io.get_regions().tolist()
countries.append('QC')
extended_exio.index = pd.MultiIndex.from_product([countries,io.get_sectors()],names=('region', 'sector'))
extended_exio.columns = pd.MultiIndex.from_product([countries,io.get_sectors()],names=('region', 'sector'))

We assume that the only average technolgoy that differs greatly between Quebec and the other provinces of Canada is the electricity generation. We will thus replace the average grid mix of Canada currently used by our Quebec economy by a new grid mix that reflects Quebec's own grid mix.

For that, we first introduce a QC grid_mix technology in the extended_exio dataframe (both row and column) that will be empty for now.

In [56]:
# new column
extended_exio[('QC','grid_mix')] = pd.Series(0, extended_exio.index)
# new row
extended_exio = pd.concat([extended_exio,pd.DataFrame(0,pd.MultiIndex.from_product([['QC'],['grid_mix']],names=('region', 'sector')),extended_exio.columns)],axis=0)

We then populate that QC grid mix technology according to existing electricity generation sectors of Exiobase. Here we consider that Quebec's grid mix is 99.1% hydro, 0.5% wind, 0.2 petrol and 0.2 biomass.

In [57]:
# you can change Quebec grid mix here if you want (give %)
grid_mix_quebec = {'Electricity by coal':0,'Electricity by gas':0,'Electricity by nuclear':0,'Electricity by hydro':99.1,'Electricity by wind':0.5,
                  'Electricity by petroleum and other oil derivatives':0.2,'Electricity by biomass and waste':0.2,'Electricity by solar photovoltaic':0,
                  'Electricity by solar thermal':0,'Electricity by tide, wave, ocean':0,'Electricity nec':0}

We introduce our newly defined QC grid mix in extended_exio

In [58]:
for technology in grid_mix_quebec.keys():
    extended_exio.loc[('QC',technology),('QC','grid_mix')] = grid_mix_quebec[technology]/100

We can see that the QC grid mix is supplied with electricity from Quebec. However, keep in mind that this electricity from quebec is just a copy of the canadien electricity. Although the grid mix has been changed, the technologies behind are considered the same, i.e., the hydro-electricity is considered the same across Canada.

In [59]:
extended_exio.loc[:,('QC','grid_mix')][extended_exio.loc[:,('QC','grid_mix')]!=0]

region  sector                                            
QC      Electricity by hydro                                  0.991
        Electricity by wind                                   0.005
        Electricity by petroleum and other oil derivatives    0.002
        Electricity by biomass and waste                      0.002
Name: (QC, grid_mix), dtype: float64

Even though we created the grid mix recipe, it is currently not used by the Quebec economy:

In [60]:
extended_exio.loc[('QC','grid_mix'),'QC']

sector
Paddy rice                                                                              0.0
Wheat                                                                                   0.0
Cereal grains nec                                                                       0.0
Vegetables, fruit, nuts                                                                 0.0
Oil seeds                                                                               0.0
Sugar cane, sugar beet                                                                  0.0
Plant-based fibers                                                                      0.0
Crops nec                                                                               0.0
Cattle                                                                                  0.0
Pigs                                                                                    0.0
Poultry                                                                  

Let's remedy that by allocating the canadian electricity used by the Quebec economy to the QC grid mix:

In [61]:
extended_exio.loc[('QC','grid_mix'),'QC'] = extended_exio.loc[[i for i in extended_exio.index if 'Electricity' in i[1] and 'CA' == i[0]],'QC'].sum().values

Now we see that the QC grid mix is used by Quebec's economy:

In [63]:
extended_exio.loc[('QC','grid_mix'),'QC']

sector
Paddy rice                                                                              1.406268e-03
Wheat                                                                                   3.711377e-03
Cereal grains nec                                                                       4.744554e-03
Vegetables, fruit, nuts                                                                 5.376579e-03
Oil seeds                                                                               3.248464e-03
Sugar cane, sugar beet                                                                  1.529783e-03
Plant-based fibers                                                                      1.559345e-03
Crops nec                                                                               1.309083e-02
Cattle                                                                                  9.884977e-03
Pigs                                                                                

Final steps, we need to deal with imports and exports of the Quebec economy.

For international trade, we could not separate commodities bought from/by Quebec from Canada. We thus assume that international trade will only occur with Canada.

In [23]:
extended_exio.loc['QC',io.get_regions()]=0

Unfortunately, we did not find any data on interprovince trade. We thus assume that there is no interprovince trade. All quebecers will thus not be able to buy Canadian products in this IO table. And, to make matters worst, since international trade is only connected to Canada, Quebecers will **only** buy products from Quebec. Remember though, that all technologies are identical between our version of Quebec's economy and Canada's economy as a whole albeit with a different grid mix.

In [69]:
extended_exio.loc['CA','QC'] = 0

We use the pickle library to store the newly extended EXIOBASE version with Quebec included. We will use that pickle in other notebooks to use the matrix we just created here.

In [26]:
with open('one_matrix_to_rule_them_all.pickle','wb') as f:
    pickle.dump(extended_exio, f)